In [10]:
# Include packages here
import uuid
import hashlib
import random
import copy

In [11]:
class Agent:
    
    # Initialize the agents here
    def __init__(self):
        self.id = uuid.uuid1()
        self.messagesReceived = []
        self.messagesToSend = []
        self.messagesQueue = []
        self.currency = 0
        
    # Agents are sending messages to other agents regarding transactions through this function
    def sendMessages(self, agents):
        
        # If there are any messages to be sent saved in this list, try to send them
        if len(self.messagesToSend) > 0:
            
            # Loop through all messages and and all agents that are part of this blockchain
            for message in self.messagesToSend:
                for agent in agents:
                    
                    # Have a random chance to successfully send a message
                    chance = random.randint(1,100)
                    
                    # This chance is set here:
                    if chance < 10:
                        agent.messagesQueue.append(copy.deepcopy(message))
    
    # In here, an agent analyzes the messages that are currently queued
    def analyseQueueMessages(self):
        
        # If there are any messages, loop through them
        if len(self.messagesQueue) > 0:
            for message in self.messagesQueue:
                
                # Check whether this agent has already received this message. 
                counter = 0
                for message2 in self.messagesReceived:
                    if message.id == message2.id and message.fromA == message2.fromA:
                        counter += 1
                
                # Store this message if it hasn't been received already
                if counter == 0:
                    self.messagesReceived.append(copy.deepcopy(message))
        
        # Empty the queue at the end
        self.messagesQueue = []
    
    # This functions checks how much money the sender has, based on the transactions 
    # done already in the blockchain by the sender
    def checkAmount(self,blockchain,agent):
        
        check = 0
        
        for block in blockchain:
            if block.sender == agent:
                check -= block.data
            elif block.receiver == agent:
                check += block.data
    
        return check
    
    # This function analyzes the truth of the received messages
    def analyseReceivedMessages(self,pending,blockchain):
        if len(self.messagesReceived) > 0:
            
            # First, we want to check all messages received, not only once, but at every "step"
            for message in self.messagesReceived:
                

                # Calculate how much money the sender has based on the transactions recorded in the blockchain
                check = self.checkAmount(blockchain,message.sender)

                # If the sender has the money mentioned in the "data" variable, mark the message as true (1)
                # Else, mark the message as false (-1)
                # NOTE: We have to calculate this every time because a transaction can be approved and a false message
                #       could transform into a true one, and vice-versa
                if check < message.data:
                    message.counter = -1
                else:
                    message.counter = 1
            
            # After we calculated all the messages' truth value, we need to check whether we have received 2 identical
            # messages (more or less), one from sender and one from receiver, containing the same information!
            for x in self.messagesReceived[:]:
                for y in self.messagesReceived[:]:
                    
                    # Safety check here (for the truth value)
                    if x.id == y.id and x.fromA != y.fromA and (x.counter == 1 or x.counter == -1):
                        
                        # Go through all pending blocks in the blockchain queue
                        for z in pending:
                            
                            # If we find an ID match, we want to increase or decrease the validation counter
                            if z.id == x.id:
                                if x.data != y.data or x.counter == -1 or y.counter == -1:
                                    z.counter -= 1
                                else:
                                    z.counter += 1

    # This function checks whether the messages received by the agent are outdated or not
    # (whether they have been added to the blockchain or not; if they did, delete the messages)
    def checkBlockchainQueue(self, pending):
        
        for aMessage in self.messagesReceived[:]:
            counter = 0
            
            for bMessage in pending:
                if aMessage.id == bMessage.id and aMessage.fromA == bMessage.fromA:
                    counter += 1
                    
            if counter == 0:
                self.messagesReceived.remove(aMessage)

In [12]:
# This is a basic block class that is used for adding blocks in the blockchain
class Block:
    def __init__(self, blockNo, pHash, data, sender, receiver):
        self.blockNo = blockNo
        self.pHash = pHash
        self.data = data
        self.sender = sender
        self.receiver = receiver
        
        self.cHash = self.computeHash(blockNo, pHash, data, sender, receiver)
        
    def computeHash(self,blockNo, pHash, data, sender, receiver):
        string = str(blockNo)+str(pHash)+str(data)+str(sender)+str(receiver)
        encoded = string.encode()
        result = hashlib.sha256(encoded)

        return result.hexdigest()

In [13]:
# This class is used for pending blocks (agents memory and blockchain queue)
class pendingBlock:
    def __init__(self, data, sender, receiver, sentBy, ID):
        self.id = ID
        self.sender = sender
        self.receiver = receiver
        self.data = data
        self.fromA = sentBy
        self.checkedBy = []
        self.counter = 0

In [14]:
# This is the actual blockchain class
class BlockChain:
    
    # We initialize here the blockchain and we are adding 100 agents to the network
    def __init__(self):
        self.blockchain = [self.genesisBlock()]
        self.pendingBlocks = []
        self.agents = []
        
        for x in range(0,100):
            self.newAgentInNetwork()
    
    # We define the genesis block here (all blockchains have a starting point where previous hash is null)
    def genesisBlock(self):
        return Block(0,0,0,0,0)
    
    # We are ading a new block to the blockchain, based on a couple of scenarios
    def addNewBlock(self,newBlock):
        
        # First, we identify the sender and receiver 
        receiver = next((x for x in self.agents if x.id == newBlock.receiver), None)
        sender = next((x for x in self.agents if x.id == newBlock.sender), None)
        
        # This part is used to initialize agents with 10000 coins to the networks, so no sender here!
        if newBlock.sender == "Blockchain":
            self.blockchain.append(copy.deepcopy(newBlock))
            
            if receiver != None:
                receiver.currency += newBlock.data

        # In here, we are adding a new block to the blockchain, only if the data is valid (double checking here)
        elif newBlock.sender != "Blockchain" and sender.currency - newBlock.data > 0 and receiver != None and sender != None:
            self.blockchain.append(copy.deepcopy(newBlock))
            
            receiver.currency += newBlock.data
            sender.currency -= newBlock.data
        
            print("New Block (with valid data) entered successfully!")
            
        # If the data is not valid, the block is going to be rejected (in case the sender doesn't actually have the money)
        # NOTE: It is important to have this here because the code runs sequentially. In the normal world, everything is 
        #       running asyncronosly, and there are other types of checks done for validation. But through this project we 
        #       are more interested in how do they communicate, rather to make sure is a 1:1 copy of an actual blockchain
        elif newBlock.sender != "Blockchain" and sender.currency - newBlock.data < 0:
            print("Block rejected! Invalid data!")
            
    
    # Through this we are adding a new agent to the network.
    # The functions should be self explanatory.
    def newAgentInNetwork(self):
        agent = Agent()
        self.agents.append(copy.deepcopy(agent))
        prevBlock = self.blockchain[len(self.blockchain)-1]
        
        block = Block(len(self.blockchain),
                     prevBlock.cHash,
                     10000,
                     "Blockchain",
                     agent.id)
        
        self.addNewBlock(block)
      
    # At every step, the validity of the blockchain is verified!
    def checkBlockchainValidity(self):
        counter = 0
        for x in range(len(self.blockchain)):
            if x > 0 and self.blockchain[x].pHash != self.blockchain[x-1].cHash:
                counter += 1
                
        if counter != 0:
            print("Blockchain Validity Check failed!!!")
    
    # All pending blocks are checked. If either 2 messages (from sender and receiver) managed to surpass the 5 or -5
    # threshold, the block (and messages) will be approved or rejected respectively.
    def checkPending(self):
        for aMessage in self.pendingBlocks[:]:
            for bMessage in self.pendingBlocks[:]:
                
                # In here, the messages/block are accepted and the block gets added to the blockchain
                if aMessage.id == bMessage.id and aMessage.fromA != bMessage.fromA and aMessage.counter > 5 and aMessage.counter == bMessage.counter and aMessage.data == bMessage.data:
                    prevBlock = self.blockchain[len(self.blockchain)-1]
                    block = Block(len(self.blockchain),
                                 prevBlock.cHash,
                                 aMessage.data,
                                 aMessage.sender,
                                 aMessage.receiver)
                    self.addNewBlock(block)
                    self.pendingBlocks.remove(aMessage)
                    self.pendingBlocks.remove(bMessage)
                    print("### Valid Block Added! " + str(aMessage.counter) + " agents validated this transaction")
                    print("############################")
                    print("Information about the added block:")
                    print("Block no: " + str(block.blockNo))
                    print("Block hash: " + block.pHash)
                    print("Block sender ID: " + str(block.sender))
                    print("Block receiver ID: " + str(block.receiver))
                    print("Block data: " + str(block.data))
                    print("############################")
                    print("")
                    
                # In here, the messages/block gets rejected and the block and messages get removed from the queue 
                elif aMessage.id == bMessage.id and aMessage.fromA != bMessage.fromA and aMessage.counter < -5 and aMessage.counter == bMessage.counter and aMessage.data != bMessage.data:
                    self.pendingBlocks.remove(aMessage)
                    self.pendingBlocks.remove(bMessage)
                    print("### Invalid Block Removed! Agents rejectected this block!")
      
    # This is the step function and you can iterate through the blockchain, having the agents sending messages
    # and approving/rejecting blocks. Use the "no" variable to set the number of steps!
    def step(self, no):
        
        for noOfSteps in range(0,no):
            # First, get a sender and a receiver from the agent list
            sender = self.agents[random.randint(0,len(self.agents)-1)]
            receiver = self.agents[random.randint(0,len(self.agents)-1)]

            # Making sure that the sender is not going to send messages to itself
            while sender == receiver:
                receiver = self.agents[random.randint(0,len(self.agents)-1)]

            # Get the chance that the messages are either different or invalid (difMsg and invalData respectively)
            difMsg = random.randint(1,100)
            invalData = random.randint(1,100)

            # Add a chance that the data sent by sender to be invalid (False)
            if invalData <= 10:
                dataSender = random.randint(1,int(1.1*sender.currency))
            else:
                dataSender = random.randint(1,sender.currency)

            # Add a chance that they are sending different messages
            if difMsg <= 10:
                dataReceiver = random.randint(1,sender.currency)
            else:
                dataReceiver = dataSender

            # Add a chance to have a new transaction. There can also be a chance that the transaction failed due to
            # whatever reason
            transaction = random.randint(1,100)
            if transaction >= 30:
                uniqueID = uuid.uuid1()
                messageSender = pendingBlock(dataSender, sender.id, receiver.id, sender.id, uniqueID)
                messageReceiver = pendingBlock(dataReceiver, sender.id, receiver.id, receiver.id, uniqueID)
                sender.messagesToSend.append(copy.deepcopy(messageSender))
                receiver.messagesToSend.append(copy.deepcopy(messageReceiver))
                self.pendingBlocks.append(copy.deepcopy(messageSender))
                self.pendingBlocks.append(copy.deepcopy(messageReceiver))

            # Iterate through all agent's messages to be sent
            for agent in self.agents:
                agent.sendMessages(self.agents)

            # Iterate through all agent's queued messages
            for agent in self.agents:
                agent.analyseQueueMessages()

            # Iterate through all agent's received messages and analyze them
            for agent in self.agents:
                agent.analyseReceivedMessages(self.pendingBlocks, self.blockchain)

            # Check the pending messages after the agents sent their validation and see if there is something that
            # needs to be added in the blockchain
            self.checkPending()
            
            # Iterate through all agent's messages to see whether they have been removed from the queue or are still
            # up-to-date
            for agent in self.agents:
                agent.checkBlockchainQueue(self.pendingBlocks)
                
            # Finally, check blockchains integrity!
            self.checkBlockchainValidity()

In [15]:
blockchain = BlockChain()

In [16]:
blockchain.step(10)

An agent found some invalid data :(
An agent found some invalid data :(
An agent found some invalid data :(
An agent found some invalid data :(
An agent found some invalid data :(
An agent found some invalid data :(
An agent found some invalid data :(
An agent found some invalid data :(
An agent found some invalid data :(
An agent found some invalid data :(
An agent found some invalid data :(
An agent found some invalid data :(
An agent found some invalid data :(
An agent found some invalid data :(
An agent found some invalid data :(
An agent found some invalid data :(
An agent found some invalid data :(
An agent found some invalid data :(
An agent found some invalid data :(
An agent found some invalid data :(
An agent found some invalid data :(
An agent found some invalid data :(
An agent found some invalid data :(
An agent found some invalid data :(
An agent found some invalid data :(
An agent found some invalid data :(
An agent found some invalid data :(
An agent found some invalid 